<a href="https://colab.research.google.com/github/JunmoJeong/credit_risk_modeling/blob/main/credit_risk_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
채무 불이행 예측
문제를 정의하고 사용 사례를 이해하고 나면 개발하기 쉽다.
금융기관은 채무자가 미래에 대출을 갚기에 충분한 돈을 지니게 될지 여부를 조사한다. 
분석 ouput은 확률이다. (채무자의 채무 불이행 가능성)
여기서 확률은 고객이 제때 대출에 대한 월별 상환액(EMI)을 지불하지 않을 가능성을 나타낸다.

높은 확률 값은 고객이 금융기관의 채무 불이행자 또는 부적합 채무자가 될 가능성을 나타낸다. 
'''

In [ ]:
'''
Dependency Libraries
Python 2.7
pandas
numpy
scipy
scikit-learn
matplotlib
seaborn
jupyter notebook

'''

# 데이터 특성


In [ ]:
'''
cs-training.csv : 훈련 데이터셋
cs-test.csv : 테스트 데이터셋
DataDictionary.xls : 데이터셋의 각 특성에 대한 정보가 들어있다. 
sampleEntry.csv : 테스트 데이터셋의 최종 출력을 생성해야 하는 형식에 대한 내용.

'''

### SeriousDlqin2yrs 
: 채무자가 지난 2년간 90일까지 기한 경과 했는지에 대한 여부
target label이다.

### RevolvingUtilizationOfUnsecuredLines
: 현재의 대출 채무 및 부동산을 제외한 채무자의 신용카드 한도
계정 보유자의 신용 잔고 및 개인 계좌 잔고를 사용
ex) 카드 한도 1000 카드 500 사용 후 잔고 500 + 1000(은행 계좌 잔고)
값은 : 1500(사용가능) / 2000(총 금액) = 0.75

### Age
: 채무자 나이

### NumberOfTime30-59DaysPastDueNotWorse
: 채무자가 늦게라도 원리금 납부 지불했지만 만기일로부터 30일이 지난 후 또는 만기일로부터 59일이 지나기 전에 지불한 횟수

### DebtRatio
: 부채 비율
ex) 월간 부채가 200, 다른 지출이 500이면 월 소득이 1000이라고 치면 
DebtRatio는 700/1000 = 0.7

### MontlyIncome
: 월간 소득

### NumberOfOpenCreditLinesAndLoans
: 마이너스 통장 개수 및 신용카드 개수

### NumberOfTimes90DaysLate
: 채무자가 자신의 원리금 납부를 하고나서 90일 후에 미지급금을 지불한 횟수

### NumberRealEstateLoansOrLines
: 채무자가 부동산 담보 대출 개수 또는 주택 담보 대출 개수

### NumberOfTimes60-89DaysPastDueNotWorse
: 채무자가 늦게 원리금을 지불햇지만 만기일로부터 60일이 지난 후 또는 만기일로부터 89일이 지나기 전에 지불한 횟수

### NumberOfDependents
: 부양가족 수(채무자 제외)



In [7]:
import pandas as pd

training_data = pd.read_csv("/content/drive/MyDrive/Junmo Jeong/Projects/credit_risk_prediction/cs-training.csv")
training_data.head()


,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [8]:
print(training_data.shape)

(150000, 12)


In [9]:
# 제일앞에 Unnamed:0 칼럼 이상하니 이름 바꿔보자
training_data.rename(columns = {"Unnamed: 0": "ID"}, inplace=True)
training_data.head()

,ID,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [10]:
# 처음에 불러올 때부터 없애보자. ID 칼럼 필요없으니
training_data = pd.read_csv("/content/drive/MyDrive/Junmo Jeong/Projects/credit_risk_prediction/cs-training.csv").drop('Unnamed: 0', axis = 1)
training_data.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [11]:
# 컬럼명 중에 - 이것들을 없애자 헷갈리니까 그리고 소문자로 통일해보자. 나중에 테스트 데이터셋에도 해야됨.
cleancolumns = []
for i in range(len(training_data.columns)):
  cleancolumns.append(training_data.columns[i].replace('-','').lower())
training_data.columns = cleancolumns

training_data.head()

,seriousdlqin2yrs,revolvingutilizationofunsecuredlines,age,numberoftime3059dayspastduenotworse,debtratio,monthlyincome,numberofopencreditlinesandloans,numberoftimes90dayslate,numberrealestateloansorlines,numberoftime6089dayspastduenotworse,numberofdependents
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0
